##### Information-theretic analysis of language models (Fall 2022/3)


# Home Assignment 1

#### Topics:
1. $N$-gram language models
2. Entropy (definition)


#### Due: 24/11/2022 by class time

#### Instructions:
- Write your name, Student ID, and date in the cell below. 
- Submit a copy of this notebook with code filled in the relevant places as the solution of coding excercises.
- For theoretic excercises, you can either write your solution in the notebook using $\LaTeX$ or submit additional notes.

<hr>
<hr>


**Name**: Elad Prager 

**Student ID**: 200865780

**Date**: 24/11/22

$
\newcommand{\Id}{{\mathbf{I}}}  
\newcommand{\SSE}{\mathsf{SSE}}
\newcommand{\SSR}{\mathsf{SSR}}
\newcommand{\MSE}{\mathsf{MSE}}
\newcommand{\simiid}{\overset{iid}{\sim}}
\newcommand{\ex}{\mathbb E}
\newcommand{\var}{\mathrm{Var}}
\newcommand{\Cov}[2]{{\mathrm{Cov}  \left(#1, #2 \right)}}
\newcommand{\one}[1]{\mathbf 1 {\left\{#1\right\}}}
\newcommand{\SE}[1]{\mathrm{SE} \left[#1\right]}
\newcommand{\reals}{\mathbb R}
\newcommand{\Ncal}{\mathcal N}
\newcommand{\abs}[1]{\ensuremath{\left\vert#1\right\vert}}
\newcommand{\rank}{\operatorname{rank}}
\newcommand{\tr}{\operatorname{Tr}}
\newcommand{\diag}{\operatorname{diag}}
\newcommand{\sign}{\operatorname{sign}}
\newcommand{\Ycal}{\mathcal Y}
\newcommand{\Xcal}{\mathcal X}
\newcommand{\Zcal}{\mathcal Z}
\newcommand{\Wcal}{\mathcal W}
$


### 1. $N$-gram model of *pride and prejudice*
You can use code from the jupyter notebook presented in class. The notebook is available on Moodle.  
The data is the content of the file `pride and prejudice.txt` available on Moodle. This file is taken from https://www.gutenberg.org/files/1342/1342-0.txt with metadata and preface removed.


1. Build an $N$-gram model using all the text of `pride and prejudice.txt` for $N=3$. 
    - How many tokens are in your vocabulary?
2. Sample and print 10 sentences from this model.
3. Evaluate perplexity; use cross-validation:
    - Split the data into roughly 10 equal parts
    - Repeat for all $i=1,\ldots,10$:
        - At step $i$, set aside part $i$ and build a model using the remaining $9$ parts.
        - Evalaute the log perplexity of the model based on the set aside part. You may skip sentences containing unseen words (remove those sentences from the average)
    - Report the averaged log perplexity over all parts
4. Say that you evaluate the log preplexity of the model in (1) under a new text created by sampling from the model as in (2). Would that log perplexity be smaller or larger than the one you found in (3)? Why? 
    - Verify your claim by estimating the perplexity of the model under at least 1,000 sentences you generated by sampling from the model you created in (1). 

## imports

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from tqdm import tqdm
from nltk import ngrams

## read dataset

In [ ]:
with open("../content/pride and prejudice.txt", 'rt') as f:
    text = f.read()
print("Number of tokens (without preprocessing) = ", len(text.split()))

Number of tokens (without preprocessing) =  124467


**Number of tokens (without preprocessing) =  124467**

In [ ]:
df = pd.read_csv("/content/pride and prejudice.txt", sep='delimiter')
df.columns = ['transcript']
print(df.describe())
print(df.head())

        transcript
count        12069
unique       12040
top     Elizabeth.
freq             5
                                          transcript
0  It is a truth universally acknowledged, that a...
1  possession of a good fortune, must be in want ...
2  However little known the feelings or views of ...
3  on his first entering a neighbourhood, this tr...
4  fixed in the minds of the surrounding families...


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


## build 3-gram model

In [ ]:
TOKEN_PATTERN = r"(?u)\b[a-zA-Z]+\b|\</?s\>"
SENT_START_TOKEN = '<s>'
SENT_END_TOKEN = '</s>'

def split_to_sentences(text: str, sep=r'\.|\n') -> str:
    return re.split(sep, text)

def is_sublist(list_a: list, list_b: list)-> bool:
    return str(list_a).strip('[').strip(']') in str(list_b)

def ng_tokenize(text: str, ng: int) -> list:
    tokens = re.findall(TOKEN_PATTERN, text.lower())
    ngz = ngrams(tokens, ng,
                 pad_left=True, pad_right=True,
                 left_pad_symbol=SENT_START_TOKEN,
                 right_pad_symbol=SENT_END_TOKEN)
    return list(ngz)

def clean_text(text: str) -> str: 
    text = text.lower()
    text = re.sub(r"<br ?/>", " ", text)  # replace '<br />' with ' '
    text = re.sub(r"\.\.+", " ", text)  # replace '..' or a longer sequence with ' '
    text = re.sub(r"-", " ", text)

    return text

def build_ngram_model(text: str, ng: int) -> pd.DataFrame:
    """
    1. Clean text
    2. Add sentence begin and end symbols
    3. Extract ngrams
    4. Remove unwanted tokens
    5. Compute frequency of every token
    
    Returns:
      dataframe. Indexes are ngrams. Columns indiacte number of occurances 
      and frequency of occurance
    """
    print("Cleaning text...")
    text = clean_text(text)
    print("Breaking to sentences")
    sentences = split_to_sentences(text)
    print("Extracting tokens...")
    tokens = []
    for sent in tqdm(sentences):
        tokens += ng_tokenize(sent, ng)
    print("Removing unacceptible tokens...")
    tokens = [t for t in tokens if not 
              (('<s>' in t[1:]) or ('</s>' in t[:-1]) or (is_sublist(["<s>", "</s>"], t)))]
    print("Counting tokens...")
    df_ng = pd.DataFrame(pd.DataFrame(tokens).value_counts()).rename(columns = {0 : 'count'})
    print("Computing frequencies...")
    df_ng.loc[:, 'freq'] = df_ng['count'] / df_ng['count'].sum()  # compute frequencies
    return df_ng

In [ ]:
model_1gram = build_ngram_model(" ".join(df.transcript), ng=1)

Cleaning text...
Breaking to sentences
Extracting tokens...


100%|██████████| 6399/6399 [00:00<00:00, 82415.76it/s]

Removing unacceptible tokens...


Counting tokens...
Computing frequencies...


In [ ]:
print(model_1gram.head(20))

      count      freq
0                    
the    4511  0.035999
to     4244  0.033868
of     3727  0.029742
and    3650  0.029128
her    2202  0.017572
i      2048  0.016343
a      2006  0.016008
in     1937  0.015458
was    1846  0.014731
she    1691  0.013495
that   1555  0.012409
it     1547  0.012345
not    1448  0.011555
you    1397  0.011148
he     1328  0.010598
his    1257  0.010031
be     1256  0.010023
as     1193  0.009520
had    1172  0.009353
with   1096  0.008746


In [ ]:
print(f'Number of tokens (after preprocessing) = {len(model_1gram)}')

Number of tokens (after preprocessing) = 6500


**Number of tokens (after preprocessing) = 6500** (My ngram model clean punctuations and dashes)

In [ ]:
model_3gram = build_ngram_model(" ".join(df.transcript), ng=3)

Cleaning text...
Breaking to sentences
Extracting tokens...


100%|██████████| 6399/6399 [00:00<00:00, 78386.18it/s]

Removing unacceptible tokens...


Counting tokens...
Computing frequencies...


In [ ]:
k = 15
print(f"{k} most frequent 3grams:")
print(model_3gram.head(k))

15 most frequent 3grams:
                        count      freq
0       1         2                    
<s>     mr        </s>    138  0.001101
        i         am       91  0.000726
        mrs       </s>     89  0.000710
        it        was      83  0.000662
of      mr        </s>     66  0.000527
i       am        sure     62  0.000495
<s>     it        is       60  0.000479
i       do        not      59  0.000471
project gutenberg tm       57  0.000455
as      soon      as       55  0.000439
she     could     not      50  0.000399
<s>     i         have     47  0.000375
        she       was      47  0.000375
and     mrs       </s>     46  0.000367
        mr        </s>     42  0.000335


## sample sentences

In [ ]:
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


def sample_from_model(ngram_model, prompt=['<s>']):
    
    def sample_from_list(df):
        return df.sample(n=1, weights = df.freq)
    
    w = ''
    state = prompt
    smp = sample_from_list(ngram_model.loc[state])
    state = list(smp.index[0][1:])
    w = list(state)
    while w[-1] != '</s>': 
        df_pool = ngram_model.loc[tuple(state)]
        smp = df_pool.sample(n=1, weights = df_pool.freq)
        state = state[1:] + [smp.index[0]]
        w.append(state[-1])
    return w[:-1] + ['</s>']

In [ ]:
for i in range(10):
  prompt = ['<s>']
  print(" ".join(prompt + sample_from_model(model_3gram, prompt=prompt)))

<s> recovering himself advanced towards the persons who so assiduously courted you </s>
<s> this part of the mother in law of wickham s a fool s errand again </s>
<s> i shall not find a very easy distance </s>
<s> collins </s>
<s> when she asked the chambermaid whether pemberley were not </s>
<s> his acquaintance with her in darcy s presence be what pin money what occasion could there be any extenuating circumstances in the habit of brooking disappointment </s>
<s> he concluded his speech with a solemn bow and though his pride never deserts him but would quit town the following morning </s>
<s> if the first place he must be greater than at home she had no turn for economy and other amiable qualification </s>
<s> if his own his aunt lady catherine de bourgh s family is indeed but considering the event must be obeyed and further apology would be spoilt by admitting a fourth </s>
<s> what shall we punish him for us to day said miss bingley with the creative eye of fancy on my poor sister 

## Evaluate perplexity

In [ ]:
class State(object):
    def __init__(self, past: list, present: list, future: list):
        self.past = past
        self.present = present
        self.future = future
        
    def step(self):
        self.past += self.present
        if len(self.future) > 0:
            self.present = [self.future.pop(0)]
        else:
            self.present = []
            self.future = []
            
    def print_state(self):
        print("past:", self.past)
        print("present:", self.present)
        print("future:", self.future)

In [ ]:
def token_probability(token : str, model: pd.DataFrame) -> float:
    if len(token) == 0:  
        return 1     # we agree that an empty token has probability 1
    token_idx = tuple(token)
    
    if token_idx in model.index:
        return model.loc[token_idx].freq.sum()  # the sum is to allow marginalization if
                                                # the token is smaller than an n-gram
    # else:
    print(f"Unrecognized Token {token}")
    raise ValueError                

def conditional_probability(token_a: list, token_b: list, model: pd.DataFrame, verbose=False) -> float:   
    pr_b = token_probability(token_b, model)
    pr_ab = token_probability(token_b + token_a, model)
    return pr_ab / pr_b
    
def sentence_probability(sent: str, model: pd.DataFrame,
                         verbose=False, backoff=False) -> float:
    ng = len(model.index[0])  # identify model order

    sent_atoms = sent.split()  
    first_token = sent_atoms[:1] 

    word_stream = State(past=[], present=first_token, future=sent_atoms[1:])

    # update state
    logprob = 0
    while len(word_stream.present) > 0:
        if backoff:
            pr_token = conditional_probability_backoff(word_stream.present,
                                                       word_stream.past[-ng+1:],
                                                       model, verbose=verbose)
        else:
            pr_token = conditional_probability(word_stream.present, word_stream.past[-ng+1:],
                                               model)
        logprob += np.log(pr_token)
        if verbose:
            word_stream.print_state()
            print(f"P(present|past) = {pr_token}")
            print("------------------------------------")
        word_stream.step()

    return np.exp(logprob)

def conditional_probability_backoff(token_a:str, token_b:str, model: pd.DataFrame, verbose=False) -> float:
    joint_token_idx = tuple(token_b + token_a)
        
    if (joint_token_idx not in model.index) and (token_b != []):
        if verbose:
            print(f"Token_a = {token_a}, Backing-off from {token_b} to {token_b[1:]}...")
        
        return conditional_probability_backoff(token_a, token_b[1:], model)
    
    return conditional_probability(token_a, token_b, model)

In [ ]:
def logperplexity(sentence: str, model: pd.DataFrame) -> float:
    N = len(sentence.split())
    prob = sentence_probability(sentence, model, backoff=True)
    return -np.log(prob) / N

In [ ]:
def preprocess_sentence(text: str) -> str:
    return  '<s> ' + " ".join(re.findall(TOKEN_PATTERN, text)).lower() + ' </s>'

def to_sentences(text: str) -> list:
    text = clean_text(text)
    sentences = split_to_sentences(text)
    return [preprocess_sentence(sent) for sent in sentences if len(sent.split()) > 2]

## KFold Model

In [ ]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 2)
iterator = iter(kf.split(df))
ls = []
for i in range(10):
  result = next(iterator)
  train = df.iloc[result[0]]
  test =  df.iloc[result[1]]
  train.columns = ['transcript']
  test.columns = ['transcript']

  model_3gram = build_ngram_model(" ".join(train.transcript), ng=3)

  test_sentences = test.transcript.tolist()
  for sent in test_sentences:
    cleaned_text = clean_text(sent)
    cleaned_text = re.sub(r"[^a-z0-9 ]+", " ", cleaned_text)
    try:
      lpp = logperplexity(cleaned_text, model_3gram)
      if lpp > 0:
          ls += [lpp]
    except:
      print("Cound not evaluate perplexity of sent: ", sent)
print(f"Average LPP of bigram model over {len(ls)} samples: {np.mean(ls)}, (std = {np.std(ls)})")

Cleaning text...
Breaking to sentences
Extracting tokens...


100%|██████████| 5738/5738 [00:00<00:00, 33380.00it/s]


Removing unacceptible tokens...
Counting tokens...
Computing frequencies...
Unrecognized Token ['mrs']
Cound not evaluate perplexity of sent:  The girls stared at their father. Mrs. Bennet said only,
Unrecognized Token ['stress']
Cound not evaluate perplexity of sent:  “Do you consider the forms of introduction, and the stress that
Unrecognized Token ['adjusting']
Cound not evaluate perplexity of sent:  “While Mary is adjusting her ideas,” he continued, “let us return
Unrecognized Token ['3']
Cound not evaluate perplexity of sent:  Chapter 3
Unrecognized Token ['mrs']
Cound not evaluate perplexity of sent:  already had Mrs. Bennet planned the courses that were to do
Unrecognized Token ['mr']
Cound not evaluate perplexity of sent:  Hurst, merely looked the gentleman; but his friend Mr. Darcy soon
Unrecognized Token ['forbidding']
Cound not evaluate perplexity of sent:  forbidding, disagreeable countenance, and being unworthy to be
Unrecognized Token ['mr']
Cound not evaluate perplexity 

100%|██████████| 5747/5747 [00:00<00:00, 69815.98it/s]

Removing unacceptible tokens...


...

100%|██████████| 5768/5768 [00:00<00:00, 67946.46it/s]


Removing unacceptible tokens...
Counting tokens...
Computing frequencies...
Unrecognized Token ['mr']
Cound not evaluate perplexity of sent:  Mr. Bennet made no answer.
Unrecognized Token ['mrs']
Cound not evaluate perplexity of sent:  “Why, my dear, you must know, Mrs. Long says that Netherfield is
Unrecognized Token ['serving']
Cound not evaluate perplexity of sent:  you do not depend on her serving you.”
Unrecognized Token ['crown']
Cound not evaluate perplexity of sent:  agreeable, and, to crown the whole, he meant to be at the next
Unrecognized Token ['starting']
Cound not evaluate perplexity of sent:  little by starting the idea of his being gone to London only to
Unrecognized Token ['circulation']
Cound not evaluate perplexity of sent:  circulation within five minutes after his entrance, of his having
Unrecognized Token ['proudest']
Cound not evaluate perplexity of sent:  character was decided. He was the proudest, most disagreeable man
Unrecognized Token ['fourth']
Cound not ev

100%|██████████| 5783/5783 [00:00<00:00, 66389.29it/s]

Removing unacceptible tokens...


Counting tokens...
Computing frequencies...
Unrecognized Token ['disclosed']
Cound not evaluate perplexity of sent:  was then disclosed in the following manner. Observing his second
Unrecognized Token ['mrs']
Cound not evaluate perplexity of sent:  Mrs. Bennet deigned not to make any reply; but, unable to contain
Unrecognized Token ['m']
Cound not evaluate perplexity of sent:  youngest, I’m the tallest.”
Unrecognized Token ['attacked']
Cound not evaluate perplexity of sent:  They attacked him in various ways; with barefaced questions,
Unrecognized Token ['generation']
Cound not evaluate perplexity of sent:  Netherfield, and leave the next generation to purchase.
Unrecognized Token ['characteristic']
Cound not evaluate perplexity of sent:  sufficiently characteristic. Bingley had never met with
Unrecognized Token ['elated']
Cound not evaluate perplexity of sent:  world. For, though elated by his rank, it did not render him
Unrecognized Token ['somehow']
Cound not evaluate perplexity of 

**The average LLP  over all parts is: 4.923**

## Evaluate perplexity for the first ngram model



**I would expect that the log perplexity found before (second Kfold model) be higher than the log perplexity in the following evaluation (first ngram model). Because in the Kfold model, the test set is truly unseen data, therefore the perplexity of the model should be higher than in the first ngram model, where the test set was generated from the train set, and the likelyhood of each sentence is higher.**

In [ ]:
ngram_sampled_sentences = []
for _ in range(1000):
  prompt = ['<s>']
  sentence = (" ".join(prompt + sample_from_model(model_3gram, prompt=prompt)))
  ngram_sampled_sentences.append(sentence)

In [ ]:
ls_sampled = []
for sent in ngram_sampled_sentences:
  try:
    lpp = logperplexity(sent, model_3gram)
    if lpp > 0:
        ls_sampled += [lpp]
  except:
    print("Cound not evaluate perplexity of sent: ", sent)
print(f"Average LPP of bigram model over {len(ls_sampled)} samples: {np.mean(ls_sampled)}, (std = {np.std(ls_sampled)})")

Average LPP of bigram model over 1000 samples: 1.8078244421510534, (std = 0.4827903389516591)


**The average LLP  over all parts is: 1.808**.

**As expected, the LLP of the first ngram model is lower than the LLP of the Kfold model**

### 2. Entropy

#### 2.1 Estimating Entropy. 
Suppose that you observe a sequence of words (or letters) $x^n \in \Xcal^n$. Assume that $x^n$ represents a sample of identically and independently distributed (IID) RVs $X^n = (X_1,\ldots,X_n)$. Namely, all $X_i$s are sampled from the same distribution and $X_i$ is sampled independently form other $X_j$ for $j\neq i$. The PMF of $X_1$ obeys the law of large numbers (e.g. it has a finite variance). Define
$$
H_n(x^n) :=  \sum_{x\in \Xcal} \frac{N(x|x^n)}{n} \log \frac{N(x|x^n)}{n},
$$
where $N(x|x^n)$ denotes the number of occurances of $x$ in the sequence $x^n$ ( $N(x|x^n) = \sum_{i=1}^n \mathbf 1\{x=x_i\}$).

Show that
$$
\lim_{n\to \infty} H_n(x^n)/n \to H(P_X)\quad \text{(in probability)},
$$
so that $H_n(x^n)/n$ is a consistent estimator of $H(P_X)$ (https://en.wikipedia.org/wiki/Consistent_estimator).

Hint: use the law of large numbers, and the continuous mapping theorem (https://en.wikipedia.org/wiki/Continuous_mapping_theorem).


**2.1 ANSWER:**

Given the definition: $H_n(x^n) :=  -\sum_{x\in \Xcal} \frac{N(x|x^n)}{n} \log \frac{N(x|x^n)}{n}$ and given that $N(x|x^n)$ denotes the number of occurances of $x$ in the sequence $x^n$.

Using the law of large numbers, we can state that as the sample size grows, its mean gets closer to the average of the whole population. Therefore, for ${n\to \infty}$, $x^n \to X^n$. 

Using the continuous mapping theorem, if $x_n \to X_n$ then $f(x_n) \to f(X_n)$. Therefore, we can say that: $\frac{N(x|x^n)}{n} \to \frac{N(x|X^n)}{n}$, which is similar to take the probability: $P_{X}(x)$ for ${n\to \infty}$.

Thus placing it in the definition gives: $\lim_{n\to \infty} H_n(x^n) \to -\sum_{x\in \Xcal} P_{X}(x) \log P_{X}(x).$

Also, the Shannon information associated with $P_X$ is: $-\sum_{x\in \Xcal} P_{X}(x) \log P_{X}(x)= H(P_X)$.

By placing it in the previous equation we get: $\lim_{n\to \infty} H_n(x^n) \to H(P_X)$.


#### 2.2 Equivalent representations of conditional entropy
(the porpuse of this question is to get you familiar with the probability notation). 

Let $X,Y$ be a pair of RVs with a joint mpf $P_{XY}$ such that $P_Y(y) > 0$ for all $y \in \Ycal$. Define:
$$
H(X|Y) :=  -\sum_{x \in \Xcal, y \in \Ycal} P_{XY}(x,y) \log_2 P_{X|Y}(x|y). 
$$

Argue that:
1. $H(X|Y) = -\sum_{y \in \Ycal} P_Y(y) \sum_{x \in \Xcal} P_{X|Y}(x|y) \log \left(P_{X|Y}(x|y)\right)$
2. $H(X|Y) = \ex[H(X|Y=Y_1)]$ where $Y_1 \in \Ycal$ is a RV with pmf $P_Y$
3. $H(X|Y) = \ex\left[ -\log\left( P_{X|Y}(X_1|Y_1) \right) \right]$ where $Y_1 \in \Ycal$ and $X_1 \in \Xcal$ are  RVs with pmf $P_{XY}$

Hint for 2 and 3: recall that for a function $f : \Xcal \to \reals$ and a RV $X \in \Xcal$ we define:
$$
\ex \left[f(X) \right] := \sum_{x \in \Xcal} P_X(x) f(x)
$$


**2.2 ANSWER:**

1.
By definition, we are given that: $H(X|Y) :=  -\sum_{x \in \Xcal, y \in \Ycal} P_{XY}(x,y) \log_2 P_{X|Y}(x|y).$

We knows from conditional probability that: $P_{X|Y}(x|y)=P_{XY}(x,y)/P_Y(y)$

We can simplify that to: $P_{XY}(x,y)=P_Y(y)*P_{X|Y}(x|y)$

Therefore, we can place the right hand side over the given first equation, 

and get: $H(X|Y) = -\sum_{x \in \Xcal, y \in \Ycal} P_Y(y)*P_{X|Y}(x|y) \log \left(P_{X|Y}(x|y)\right)$

and by seperating the sums, we can simplify that to:

$H(X|Y) = -\sum_{y \in \Ycal} P_Y(y) \sum_{x \in \Xcal} P_{X|Y}(x|y) \log \left(P_{X|Y}(x|y)\right)$

2.
We Knows from conditional probability that: $H(X|Y) :=  \sum_{y \in \Ycal} P_{Y}(y) H(X|Y=y).$

We also knows from the hint that: $\ex \left[f(X) \right] := \sum_{x \in \Xcal} P_X(x) f(x)$

Specifically in our case: $\ex(H(X|Y=y)) := \sum_{y \in \Ycal} P_{Y}(y)H(X|Y=y)$

Therefore, we can place it in the right hand side of first equation, and get: 

$H(X|Y) :=  \ex(H(X|Y=Y_1)]$ where $Y_1 \in \Ycal$ is a RV with pmf $P_Y$



3.
By definition, we are given that: $H(X|Y) :=  -\sum_{x \in \Xcal, y \in \Ycal} P_{XY}(x,y) \log P_{X|Y}(x|y).$

We also knows from the hint that: $\ex \left[f(X) \right] := \sum_{x \in \Xcal} P_X(x) f(x)$

Specifically in our case: $\ex \left[-\log_2 P_{X|Y}(x|y) \right] := -\sum_{x \in \Xcal, y \in \Ycal} P_{XY}(x,y) \log_2 P_{X|Y}(x|y)$

Therefore, we can place the left hand side over the given first equation, and get: 

$H(X|Y) = \ex\left[ -\log\left( P_{X|Y}(X_1|Y_1) \right) \right]$ where $Y_1 \in \Ycal$ and $X_1 \in \Xcal$ are  RVs with pmf $P_{XY}$